# A brief summary about generic artificial neural network.
### __1. What is a neural network?__

Complex as it may seem, a neural network is just a function that takes a set of inputs and returns a set of outputs.  
We can define a neural network as follows:  

$\hat{y} = f(\varphi({W}^{(L)}\varphi({W}^{(L-1)}...\varphi({W}^{(1)}x))))$, that is:

Given a set of inputs $x$, the neural net use linear mapping to map themn to a set $z^{1}$ ($z^{1} = Wx$, however we often add a $1$ to $x$ to represent the bias), then apply the activation function $\varphi$ to get $a^{1}$ ($a^{1} = \varphi{(z^{1})}$), and then linearly maps $a^{1}$ to a set $z^{2}$, and so on until we get $z^{(L)}$. At this point it will use the function $f$ to map $z^{(L)}$ to a set of outputs $y$. Note that most of the time $f$ and $\varphi$ are not the same.
- $f$ is the activation function at the output layer
- $\varphi$ is the activation function at the hidden layers.  
- $x = \begin{pmatrix}x_1\\x_2\\.\\.\\. \end{pmatrix}$ is the input

- $\hat{y} = \begin{pmatrix}\hat{y}_1\\\hat{y}_2\\.\\.\\. \end{pmatrix}$ is the output
- $L \in N^*$ is the number of layers
- ${W}^{(l)} = \begin{pmatrix}w^{(l)}_{00}&w^{(l)}_{01}&.&.&.&w^{(l)}_{0m}\\w^{(l)}_{10}&w^{(l)}_{11}&.&.&.&w^{(l)}_{1m}\\.&.&.&&&.\\.&.&&.&&.\\.&.&&&.&.\\w^{(l)}_{n0}&w^{(l)}_{n1}&.&.&.&w^{(l)}_{nm} \end{pmatrix}$ is the weight matrix from layer $(l - 1)$ to layer $(l)$

where:

* $w^{(l)}_{ij}$ is the weight from the $j^{th}$ neuron in layer $(l - 1)$ to the $i^{th}$ neuron in layer $(l)$

* $m$ is the number of nodes in layer $(l - 1)$

* $n$ is the number of nodes in layer $(l)$

### __2. Computing the gradient__

A neural net is really just a fairly complex/nested function. Our goal when saying "Training a neural network" is to find the best set of weights to get the value $\hat{y}$ as close as possible to the target value $y$ and use it to predict the value of $y$ given the value of $x$. More precisely, we want to find the set of weights that minimizes the following loss function (also called the cost function):

$C = \sum_{i=1}^{N} \frac{1}{2} \left( y_i - \hat{y}_i \right)^2 ~~~~~ \text{for regression problems}$

$C = \sum_{i=1}^{N} -y.\log{\hat{y}} - (1 - y).\log(1-\hat{y}) ~~~~~ \text{for classification problems}$

where $N$ is the number of samples.

A common way to minimize this loss function is to use [gradient descent](https://towardsdatascience.com/gradient-descent-algorithm-a-deep-dive-cf04e8115f21#:~:text=Gradient%20descent%20(GD)%20is%20an,e.g.%20in%20a%20linear%20regression). However, gradient descent (and most other algorithms) require that we know the gradient of the loss function with respect to the weights: $\nabla_{W}^{(l)} C$ where $l = 1, 2, .., L$, that is we need to evaluate $\frac{\partial{C}}{\partial{w^{(l)}_{ij}}}$ for every $w^{(l)}_{ij}$ belonging to $W^l$ for every layer $l$.

So how can we do that? One way is, of course, just directly evaluate it! But this approach has some problems:
1. The function of the neural network ($\hat{y} = f(...)$) is very complex. Doing this directly is not easy.
2. IF you've tried to do this before, you'll see that there are a lot of redundant computations since many values are computed multiple times.

Another approach is to use the backpropagation algorithm: we will start by computing the loss in terms of the output layer and then use chain rule to backwardly compute the loss in terms of other layers:

In this section, if you encounter $m$ and $n$ in the same equation, $m$ should be the number of nodes in the previous layer and $n$ should be the number of nodes in the next layer. For example: $m$ can be the number of nodes in layer $(l - 1)$ and $n$ can be the number of nodes in layer $(l)$ or $m$ can be the number of nodes in layer $(l)$ and $n$ can be the number of nodes in layer $(l + 1)$.

Computing the loss in terms of the output is just a straightforward computation since $C$ is a function of $\hat{y}$. After doing this, we obtain $\nabla_{\hat{y}} C$.

Now we should work backwards:

We have that $\frac{\partial{C}}{\partial{w_{ij}^{(l)}}} = \sum_{k=0}^{n}\frac{\partial{C}}{\partial{z^{(l)}_{k}}} \frac{\partial{z^{(l)}_{k}}}{\partial{w_{ij}^{(l)}}}$.

But $\frac{\partial{z^{(l)}_{k}}}{\partial{w_{ij}^{(l)}}} = \frac{\partial{\sum_{t=0}^{m}w_{kt}^{(l)}a^{(l-1)}_{t}}}{\partial{w_{ij}^{(l)}}} = a_{j}$ if $i = k$ and $0$ otherwise.

$\Rightarrow \frac{\partial{C}}{\partial{w_{ij}^{(l)}}} = a_{j}^{(l-1)} \frac{\partial{z^{(l)}_{i}}}{\partial{w_{ij}^{(l)}}}$

$\Rightarrow \frac{\partial{C}}{\partial{w^{(l)}}} = \frac{\partial{C}}{\partial{z^{(l)}}}(a^{(l-1)})^T$

Let $\delta^{(l)} = \begin{pmatrix} \frac{\partial{C}}{\partial{z^{(l)}_{k}}}\\\frac{\partial{C}}{\partial{z^{(l)}_{k}}}\\.\\.\\. \end{pmatrix}$ (called the error at  layer $l$)

$\Rightarrow \frac{\partial{C}}{\partial{w^{(l)}}} = \delta^{(l)} a^{(l-1)T}$

Almost there! Only $\delta^{(l)}$ is not yet computed:

At the output layer, we've already computed $\delta^{(L)} = \nabla_{\hat{y}} C$.

At other layers:

$\frac{\partial{C}}{\partial{z^{(l)}_{i}}} = \sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}\frac{\partial{z^{(l+1)}_{j}}}{{\partial{z^{(l)}_{j}}}}) = \sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}\frac{\partial{\sum_{k=0}^{m}w_{jk}^{(l+1)}a_{k}^{(l)}}}{\partial{z_{i}^{(l)}}})$

In $\sum_{k=0}^{m}w_{jk}^{(l+1)}a_{k}^{(l)}$, there is only 1 term that depends on $z_{i}^{(l)}$ which is $w_{ji}^{l+1}a_{i}^{(l)}$ and its derivative with respect to $z_{i}^{(l)}$ is $w_{ji}^{(l+1)}\varphi'{(z_{i}^{(l)})}$.

$\Rightarrow \frac{\partial{C}}{\partial{z^{(l)}_{i}}} = \sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{ji}^{(l+1)}\varphi'{(z_{i}^{(l)})})$

$\delta^{(l)} = \begin{pmatrix} \sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{j0}^{(l+1)}\varphi'{(z_{0}^{(l)})})\\\sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{j1}^{(l+1)}\varphi'{(z_{1}^{(l)})})\\.\\.\\. \end{pmatrix} = \begin{pmatrix} \sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{j0}^{(l+1)})\\\sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{j1}^{(l+1)})\\.\\.\\. \end{pmatrix} \circ \begin{pmatrix}\varphi'{(z_{0}^{(l)})}\\\varphi'{(z_{1}^{(l)})}\\.\\.\\. \end{pmatrix}$

where $\circ$ is the Hadamard product a.k.a element-wise product.

Moreover: 

$\begin{pmatrix} \sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{j0}^{(l+1)})\\\sum_{j=0}^{n}(\frac{\partial{C}}{\partial{z^{(l+1)}_{j}}}w_{j1}^{(l+1)})\\.\\.\\. \end{pmatrix} = \begin{pmatrix}w^{(l+1)}_{00}&w^{(l+1)}_{01}&.&.&.&w^{(l+1)}_{0n}\\w^{(l+1)}_{10}&w^{(l+1)}_{11}&.&.&.&w^{(l+1)}_{1n}\\.&.&.&&&.\\.&.&&.&&.\\.&.&&&.&.\\w^{(l+1)}_{m0}&w^{(l+1)}_{n1}&.&.&.&w^{(l+1)}_{mn} \end{pmatrix} \begin{pmatrix} \frac{\partial{C}}{\partial{z^{(l+1)}_{0}}}\\\frac{\partial{C}}{\partial{z^{(l+1)}_{1}}}\\.\\.\\. \end{pmatrix} = (W^{(l+1)})^{T}\delta^{(l+1)}$

$\Rightarrow \delta^{(l)} = \varphi'(z^{(l)}) \circ (W^{(l+1)})^{T}\delta^{(l+1)}$

All set! To summarize, we have:

$\nabla_{W^{l}}C = \frac{\partial{C}}{\partial{w^{(l)}}} = \frac{\partial{C}}{\partial{z^{(l)}}}(a^{(l-1)})^T$

$\delta^{(l)} = \varphi'(z^{(l)}) \circ (W^{(l+1)})^{T}\delta^{(l+1)}$

Oh wait we still have 1 more step: compute $\varphi'(z^{(l)})$ and $a^{(l)}$ at each layer. To do this, we will perform a forward pass through the network: given a sample $x$, we will compute the activations $a^{(l)}$ and $\varphi'(z^{(l)})$ for each layer $l$ starting from the input layer and cache them to use latter.

### __3. Training the network__

There are several ways to train a neural network. One of them is to use gradient descent: we loop over the whole training set $N$ times, sum up the gradients corresponding to each sample, and then update the weights by subtracting the learning rate $\eta$ times the sum of the gradients.

However this approach is quite expensive as it requires to compute the gradients for each sample $N$ times and we don't really know how many times we will loop for the model to converge.

Another way is to use stochastic gradient descent: we loop over the whole training set $N$ times, but instead of computing the gradients for every sample, we will randomly select a sample $x_k$ and update the weights by subtracting the learning rate $\eta$ times the gradient with respect to $x_k$. Though this approach is more efficient, it is not as accurate as the gradient descent approach since it is not based on the whole training set, which makes it harder to converge.